In [ ]:
!pip install transformers

In [2]:
from tqdm import tqdm
import re
import datetime
import numpy as np



def clean(text):
  clean_tweet = re.sub("@[A-Za-z0-9_]+","", text)
  clean_tweet = re.sub("#[A-Za-z0-9_]+","", clean_tweet)
  clean_tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', clean_tweet)
  clean_tweet = re.sub(r'\d', '', clean_tweet)
  return clean_tweet

def convert(date_time):
  date_time = date_time.replace('+0000', "")
  format = '%a %b %d %H:%M:%S %Y'
  datetime_str = datetime.datetime.strptime(date_time, format)

  return datetime_str

In [4]:
from transformers import pipeline
import statistics

def mention_vectorization(data):

  classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
  number_of_mention_per_profile = 800
  output = {}

  print("Mention Vectorization")
  for item in data.items():
    prof_name = item[0].replace(".json", "")
    features = item[1]
    emo = []
    total_counter = 0

    for mention in tqdm(features.items()):
      mention_text = mention[1]
      text = clean(mention_text)
      text = text.lower()

      vector = []
      prediction = classifier(text, )
      for emotion in prediction[0]:
        vector.append(emotion['score'])

      emo.append(vector)
      total_counter += 1
      if total_counter == number_of_mention_per_profile:
        break

    temp = [[statistics.mean(k) for k in zip(*emo)]]
    output[prof_name] = temp

  return output


In [5]:
import json

def data_vectorization(mentions):

  mention = mention_vectorization(mentions)
  json_obj = json.dumps(mention, indent=4)
  with open("Textual features\\Mentions\\mentions.json", 'w', encoding="utf-8") as f:
    f.write(json_obj)
  f.close()

  return mention

In [6]:
import json

mention_path = "Labeled Data\\user_mention.json"
user_mention = {}

with open(mention_path, "r", encoding="utf8") as f:
  temp = json.load(f)
  user_mention = {x: v for x, v in temp.items()}
  f.close()

In [ ]:
mention = data_vectorization(user_mention)